In [1]:
from get_dataset import * 
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import models, layers

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-17 18:27:14.605346: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def my_map(x, y):
    return x[..., tf.newaxis], y

def my_filter(x, y):
    return y < 10

ds_train = get_dataset("train").filter(my_filter).map(my_map).batch(64)
ds_test = get_dataset("test").filter(my_filter).map(my_map).batch(64)

2023-04-17 18:27:18.363016: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7540 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:01:00.0, compute capability: 8.6


In [3]:
input_shape = next(iter(ds_train))[0].shape
input_shape

2023-04-17 18:27:24.120387: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-04-17 18:27:24.120590: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


TensorShape([64, 61, 101, 1])

In [4]:
model = models.Sequential([
    layers.Input(shape=[61, 101, 1]),
    layers.Conv2D(32, 3, activation='relu'),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(12),
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 59, 99, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 57, 97, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 28, 48, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 28, 48, 64)        0         
                                                                 
 flatten (Flatten)           (None, 86016)             0         
                                                                 
 dense (Dense)               (None, 128)               11010176  
                                                        

In [5]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

In [6]:
EPOCHS = 10
history = model.fit(
    ds_train,
    validation_data=ds_test,
    epochs=EPOCHS,
    callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=4),
)

Epoch 1/10


2023-04-17 18:27:54.504250: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-04-17 18:27:55.368864: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-04-17 18:27:55.887383: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-04-17 18:27:55.888806: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7fec78bf7200 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-04-17 18:27:55.888821: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2023-04-17 18:27:55.891626: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_uti

    290/Unknown - 6s 10ms/step - loss: 2.4626 - accuracy: 0.2317

2023-04-17 18:28:00.082922: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


290/290 [==============================] - 6s 11ms/step - loss: 2.4626 - accuracy: 0.2317 - val_loss: 2.4230 - val_accuracy: 0.1005
Epoch 2/10
290/290 [==============================] - 3s 9ms/step - loss: 2.0187 - accuracy: 0.2830 - val_loss: 2.6228 - val_accuracy: 0.2111
Epoch 3/10
290/290 [==============================] - 3s 9ms/step - loss: 1.5501 - accuracy: 0.4697 - val_loss: 1.9885 - val_accuracy: 0.3187
Epoch 4/10
290/290 [==============================] - 3s 9ms/step - loss: 1.3411 - accuracy: 0.5388 - val_loss: 1.9846 - val_accuracy: 0.3716
Epoch 5/10
290/290 [==============================] - 3s 9ms/step - loss: 1.1921 - accuracy: 0.6001 - val_loss: 1.7901 - val_accuracy: 0.4425
Epoch 6/10
290/290 [==============================] - 3s 9ms/step - loss: 1.0240 - accuracy: 0.6467 - val_loss: 1.6729 - val_accuracy: 0.4803
Epoch 7/10
290/290 [==============================] - 3s 9ms/step - loss: 0.9366 - accuracy: 0.6797 - val_loss: 1.5321 - val_accuracy: 0.5162
Epoch 8/10
290/2